# **INFO5731 In-class Exercise 2**

The purpose of this exercise is to understand users' information needs, and then collect data from different sources for analysis by implementing web scraping using Python.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission. , and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)
Describe an interesting research question (or practical question or something innovative) you have in mind, what kind of data should be collected to answer the question(s)? Specify the amount of data needed for analysis. Provide detailed steps for collecting and saving the data.

In [14]:
'''The research question that I considered is "How do the publication trends in artificial intelligence (AI) differ between various newspapers?"
The data collected in this is Title, content, publication date, author, and source (news outlet).
Articles specifically focused on artificial intelligence (AI), its technologies, ethics, impact on society, and industries

The amount of data required for this is upto 1000to 2000 samples of data.
The steps used for collecting and saving data is:
1. Identify the data sources
2. Define the query required
3. Define the data collection plan that may be done by using various methods that may be beautiful soup etc.,
4. Data storage and saving of the data.

the data is generally stored in the csv files as required.'''

'The research question that I considered is "How do the publication trends in artificial intelligence (AI) differ between various newspapers?"\nThe data collected in this is Title, content, publication date, author, and source (news outlet).\nArticles specifically focused on artificial intelligence (AI), its technologies, ethics, impact on society, and industries\n\nThe amount of data required for this is upto 1000to 2000 samples of data.\nThe steps used for collecting and saving data is:\n1. Identify the data sources\n2. Define the query required\n3. Define the data collection plan that may be done by using various methods that may be beautiful soup etc.,\n4. Data storage and saving of the data.\n\nthe data is generally stored in the csv files as required.'

## Question 2 (10 Points)
Write Python code to collect a dataset of 1000 samples related to the question discussed in Question 1.

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from collections import Counter
from textblob import TextBlob
base_url = 'https://timesofindia.indiatimes.com/'
search_query = 'artificial intelligence'
total= 1000
a_collected = 0
a_list = []

# Keywords to track
kwords = ['AI', 'ethics', 'machine learning', 'automation']

def fetch_article_links(search_url, num_pages):
    article_links = []
    for page in range(1, num_pages + 1):
        params = {'q': search_query, 'page': page}
        response = requests.get(search_url, params=params)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find article links (modify selector based on actual site)
            links = soup.find_all('a', href=True)
            for link in links:
                href = link['href']
                if 'article' in href:
                    full_url = href if href.startswith('http') else f"https://timesofindia.indiatimes.com{href}"
                    article_links.append(full_url)
        else:
            print(f"Failed to retrieve page {page}. Status code: {response.status_code}")

    return article_links

def analyze_sentiment(text):
    """Returns sentiment polarity: positive, negative, or neutral"""
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

def count_keywords(text, keywords):
    """Returns a dictionary of keyword frequencies in the text"""
    word_list = text.lower().split()
    word_counts = Counter(word_list)
    keyword_counts = {keyword: word_counts.get(keyword.lower(), 0) for keyword in keywords}
    return keyword_counts

def scrape_article(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            title = soup.find('h1').get_text() if soup.find('h1') else 'No Title'
            date = soup.find('time')['datetime'] if soup.find('time') else 'No Date'
            content = soup.find('div', class_='article-body').get_text() if soup.find('div', class_='article-body') else 'No Content'
            sentiment = analyze_sentiment(content)
            keyword_freq = count_keywords(content, kwords)

            return {'title': title, 'date': date, 'content': content, 'url': url, 'sentiment': sentiment, **keyword_freq}
        else:
            print(f"Failed to retrieve article at {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching article {url}: {e}")
        return None

def main():
    global articles_collected
    num_pages = 10  # Number of search result pages to scrape

    print("Fetching article links...")
    article_links = fetch_article_links(base_url, num_pages)
    print(f"Found {len(article_links)} article links.")

    print("Scraping articles...")
    for link in article_links:
        if articles_collected >= total_articles:
            break

        article = scrape_article(link)
        if article:
            a_list.append(article)
            articles_collected += 1
            print(f"Collected {articles_collected}/{total_articles} articles")

        time.sleep(1)  # Be respectful and avoid overwhelming the server

    print("Saving data...")
    df = pd.DataFrame(a_list)
    df.to_csv('articles.csv', index=False)
    print(f"Data saved to articles.csv")

if __name__ == "__main__":
    main()

Fetching article links...
Found 2310 article links.
Scraping articles...
Saving data...
Data saved to articles.csv


## Question 3 (10 Points)
Write Python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "XYZ". The articles should be published in the last 10 years (2014-2024).

The following information from the article needs to be collected:

(1) Title of the article

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [15]:
!pip install beautifulsoup4 requests


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def fetch_academic_news(query, start_year, end_year, total_articles):
    articles = []
    base_url = 'https://www.semanticscholar.org/blog'
    offset = 0
    rows_per_request = 10  # Adjust based on the website's pagination

    while len(articles) < total_articles:
        response = requests.get(base_url, params={'query': query, 'start': offset, 'limit': rows_per_request})
        soup = BeautifulSoup(response.text, 'html.parser')

        news_items = soup.find_all('div', class_='post-item')  # Update the selector based on actual HTML

        for item in news_items:
            title = item.find('h2', class_='title').text.strip() if item.find('h2', class_='title') else 'N/A'
            date_str = item.find('time', class_='date').text.strip() if item.find('time', class_='date') else 'N/A'
            try:
                date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                year = date_obj.year
            except ValueError:
                year = 'N/A'

            if year == 'N/A' or int(year) < start_year or int(year) > end_year:
                continue

            authors = item.find('span', class_='authors').text.strip() if item.find('span', class_='authors') else 'N/A'
            abstract = item.find('p', class_='abstract').text.strip() if item.find('p', class_='abstract') else 'N/A'
            venue = item.find('span', class_='venue').text.strip() if item.find('span', class_='venue') else 'N/A'

            articles.append([title, venue, year, authors, abstract])

            if len(articles) >= total_articles:
                break

        offset += rows_per_request
        if len(news_items) < rows_per_request:
            break  # Exit loop if no more items are returned

    return articles

# Parameters for scraping
query = "artificial intelligence"
start_year = 2014
end_year = 2024
total_articles = 1000

# Fetch articles
articles = fetch_academic_news(query, start_year, end_year, total_articles)

# Save to CSV
df = pd.DataFrame(articles, columns=['Title', 'Venue', 'Year', 'Authors', 'Abstract'])
df.to_csv("academic_ai_news_articles.csv", index=False)

print("Data collection completed. Articles are saved in 'academic_ai_news_articles.csv'.")


Data collection completed. Articles are saved in 'academic_ai_news_articles.csv'.


## Question 4A (10 Points)
Develop Python code to collect data from social media platforms like Reddit, Instagram, Twitter (formerly known as X), Facebook, or any other. Use hashtags, keywords, usernames, or user IDs to gather the data.



Ensure that the collected data has more than four columns.


In [15]:
# write your answer here
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.8 MB/s eta 0:00:00


In [4]:
import praw
import pandas as pd

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id='H2pzWfXFrWmjPSy63_8z3A',
    client_secret='SBHhAkSF7xn8G2miJrwlNsy3PrQWvg',
    user_agent='Siva'
)
# Function to collect data from Reddit
def collect_reddit_data(subreddit_name, keyword, limit=100):
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.search(keyword, limit=limit)

    data = []
    for post in posts:
        data.append({
            'title': post.title,
            'score': post.score,
            'id': post.id,
            'url': post.url,
            'created': post.created_utc,
            'num_comments': post.num_comments
        })

    return pd.DataFrame(data)

# Collect data
df_reddit = collect_reddit_data('python', 'data science', 10)
print(df_reddit)

# Save to a CSV file
df_reddit.to_csv('reddit_data.csv', index=False)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



                                               title  score       id  \
0  I teach Python courses - here's my collection ...   2969   jii8ex   
1  Six months into Python and Data science, my fi...   1731   g5ymoy   
2  Modern alternatives to Data Science Libraries ...    211  196jbms   
3  If you're a beginner interested in data scienc...    827   xyyj9t   
4  If you're a beginner interested in data scienc...    701  12j68f7   
5  1 year ago I started building Practice Probs -...    782   zzv4zt   
6  Python Data Science December [Completed] - 24 ...    517   zu7vqp   
7  78 Python data science practice problems in a ...    777   u77fce   
8  Build a Data Science SaaS App with Just Python...    102  173qcwe   
9  What features of the Python language predestin...     78   zu8azk   

                                                 url       created  \
0      https://marko-knoebl.github.io/slides/#python  1.603731e+09   
1                    https://v.redd.it/mlqov8dbicu41  1.587551e+09 

## Question 4B (10 Points)
If you encounter challenges with Question-4 web scraping using Python, employ any online tools such as ParseHub or Octoparse for data extraction. Introduce the selected tool, outline the steps for web scraping, and showcase the final output in formats like CSV or Excel.



Upload a document (Word or PDF File) in any shared storage (preferably UNT OneDrive) and add the publicly accessible link in the below code cell.

Please only choose one option for question 4. If you do both options, we will grade only the first one

In [ ]:
# write your answer here


# Mandatory Question

**Important: Reflective Feedback on Web Scraping and Data Collection**



Please share your thoughts and feedback on the web scraping and data collection exercises you have completed in this assignment. Consider the following points in your response:



Learning Experience: Describe your overall learning experience in working on web scraping tasks. What were the key concepts or techniques you found most beneficial in understanding the process of extracting data from various online sources?



Challenges Encountered: Were there specific difficulties in collecting data from certain websites, and how did you overcome them? If you opted for the non-coding option, share your experience with the chosen tool.



Relevance to Your Field of Study: How might the ability to gather and analyze data from online sources enhance your work or research?

**(no grading of your submission if this question is left unanswered)**

In [ ]:
'''
Web Scrapping is very new for me.I may need more detailed explaination if you could
explain more detailed that would be very helpful. It is very great learning new things with each other. Thank you.
'''